# Projeto: Classificação Inteligente de Feedbacks de Concessionários (Projeto Real)

**Autor:** Nathan Martins Fagundes
**Tecnologia:** Python, Pandas, Google Gemini API (LLM)

### Objetivo:
Transformar dados não estruturados (comentários de encerramento de alertas técnicos) em KPIs de gestão.
O processo resolve o problema de "Dark Data", onde milhares de feedbacks textuais eram subutilizados por inviabilidade de leitura manual (projeto real possuia mais de 5000 comentários).

### Arquitetura da Solução:
1.  **Fase 1 (Descoberta):** Utiliza uma amostra dos dados para que a IA identifique padrões e crie as categorias de negócio automaticamente.
2.  **Fase 2 (Classificação em Lotes):** Implementa uma lógica de processamento em *batch* (lotes) para classificar o dataset completo, otimizando o uso da API e contornando limitações de taxa (Rate Limit).

In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata

try:
    # Tenta pegar do segredo do Colab
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except ImportError:
    # Se estiver rodando localmente, tenta pegar de variável de ambiente ou pede input
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        GOOGLE_API_KEY = input("Insira sua Google API Key: ")

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
import pandas as pd
import json
import time
import random
from tqdm.auto import tqdm
from IPython.display import display

In [ ]:
# Criando dados fictícios para proteger a confidencialidade da CNH Industrial
# Estes dados simulam os padrões textuais reais encontrados no projeto original.

comentarios_ficticios = [
    "Cliente não atendeu as ligações, sem retorno.",
    "Falha solucionada após troca do sensor de temperatura.",
    "Monitoramento realizado, parâmetros normalizados.",
    "Alarme falso, máquina operando dentro da faixa normal.",
    "Cliente ciente, agendou visita técnica para a próxima semana.",
    "Sem falha encontrada no local, apenas sujeira no conector.",
    "Aguardando peça chegar da fábrica para reparo.",
    "Operador relatou erro de manuseio, instruído e resolvido.",
    "Tentativa de contato sem sucesso, encerrado por falta de retorno.",
    "Reset do módulo resolveu a inconsistência."
]

# Geramos 100 registros para demonstração (o projeto original usou +5.000)
dados = {
    'UniqueInstanceId': [f'MOCK-{i:04d}' for i in range(100)],
    'Quick_closure_comment': [random.choice(comentarios_ficticios) for _ in range(100)]
}

df = pd.DataFrame(dados)

In [ ]:
df_final = df[["UniqueInstanceId", "Quick_closure_comment"]].copy()
df_final.rename(columns={"Quick_closure_comment": "feedback"}, inplace=True)

print(f"Dataset de Demonstração gerado com {len(df_final)} linhas.")
df_final.head()

# --- FASE 1: DESCOBERTA DE PADRÕES (Zero-Shot / Few-Shot) ---
### Objetivo: Permitir que o LLM analise uma amostra e defina categorias baseadas na realidade dos dados, evitando viés humano prévio.


In [ ]:
sample_size = min(200, len(df_final))
amostra_comentarios = df_final['feedback'].dropna().sample(sample_size, random_state=42).tolist()
comentarios_para_analise = "\n".join([f"- {c}" for c in amostra_comentarios])

In [ ]:
resposta = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f""" Você é um analista de dados sênior. Sua tarefa é analisar a lista de comentários (feedbacks)
    de concessionários que trabalham com máquinas de construção abaixo para identificar 5 categorias. As categorias devem ter no MÁXIMO duas palavras,
    em letra minúscula e sem acentos (ex: 'falso positivo', 'peça trocada', 'falha solucionada', 'sem retorno', 'monitoramento').

**Lista de Comentários para Análise:**
{comentarios_para_analise}

**Instruções:**
1. Leia todos os comentários para entender os padrões.
2. Crie um nome de categoria para cada padrão.
3. Escreva uma descrição curta para cada categoria.
4. Retorne sua resposta ESTRITAMENTE no seguinte formato JSON:
{{
  'Nome da Categoria 1': 'Descrição da categoria 1.',
  'Nome da Categoria 2': 'Descrição da categoria 2.'
}}

    """
)

# Limpa e converte a resposta JSON
resposta_limpa = resposta.text.strip().replace("```json", "").replace("```", "")
categorias_ia = json.loads(resposta_limpa)

print("\nCategorias definidas pela IA com sucesso!")
display(categorias_ia)


# --- FASE 2: CLASSIFICAÇÃO EM LARGA ESCALA ---
###Atribuição de uma das categorias criadas a cada um dos comentários/feedbacks proporcionados pelo concessionário.

In [ ]:
categorias_para_prompt = "\n".join([f"- **{nome}**: {desc}" for nome, desc in categorias_ia.items()])



# Define o tamanho do lote (quantas linhas processar por vez)
# No projeto original (5.000 linhas), usamos batch_size = 50 para otimizar requisições.
# Para esta demonstração com 100 linhas, usaremos 10.
batch_size = 10
# Lista para guardar todos os resultados
resultados_finais = []

# tqdm vai nos dar uma barra de progresso sobre os LOTES
for i in tqdm(range(0, len(df_final), batch_size), desc="Processando Lotes"):

    # Pega uma fatia (batch) do DataFrame
    batch_df = df_final[i:i+batch_size]

    # Converte apenas esse lote para uma string JSON
    batch_json_str = batch_df.to_json(orient="records", indent=2)

    # Cria um prompt que pede para processar uma LISTA
    prompt_lote = f"""
    Sua tarefa é classificar uma LISTA de feedbacks de concessionários sobre alertas técnicos encerrados em máquinas de construção.
    Para cada objeto na lista, adicione uma chave 'categoria' baseada na sua análise.

    **Categorias Disponíveis para usar:**
    {categorias_para_prompt}

    **Regras:**
    1. Analise cada feedback individualmente.
    2. Adicione a chave 'categoria' a cada objeto JSON.
    3. Retorne a lista COMPLETA de objetos JSON, agora com a chave 'categoria'. Sua resposta deve ser APENAS o JSON.

    **Lista de Feedbacks para Processar:**
    {batch_json_str}
    """

    try:
        response = client.models.generate_content(model="gemini-2.5-flash",contents = prompt_lote)
        # Limpa e converte a resposta JSON
        resposta_limpa = response.text.strip().replace('```json', '').replace('```', '')
        # Adiciona os resultados deste lote à lista final
        resultados_finais.extend(json.loads(resposta_limpa))

    except (json.JSONDecodeError, Exception) as e:
        print(f"\nERRO ao processar o lote que começa na linha {i}. Este lote será pulado. Erro: {e}")
        # Adicionar marcadores de erro para as linhas do lote que falhou (em uma base de dados maior custa ter mais erros)
        for index, row in batch_df.iterrows():
            resultados_finais.append({
                'UniqueInstanceId': row['UniqueInstanceId'],
                'feedback': row['feedback'],
                'categoria': 'erro_no_lote'
            })

    # Pausa entre os LOTES para respeitar a quota da API
    time.sleep(4)


In [ ]:
print("\nProcesso de classificação concluído!")
df_categorizado = pd.DataFrame(resultados_finais)

In [ ]:
print("\nAmostra dos dados finais categorizados:")
display(df_categorizado.head())

In [ ]:
print("\nContagem final por categoria:")
display(df_categorizado['categoria'].value_counts())

In [ ]:
#Exportando no formato csv
output_file_name = 'Dados_Categorizados_5k.csv'
df_categorizado.to_csv(output_file_name, index=False)

In [ ]:
#Exportando no formato xlsx
output_file_name = 'Dados_Categorizados_excel.xlsx'
df_categorizado.to_excel(output_file_name, index=False)